_2021.12.01_  
[参考网址](https://docs.astropy.org/en/stable/coordinates/matchsep.html)  
* 3D matching见文档

In [23]:
import astropy.units as u
from astropy.coordinates import SkyCoord

import pandas as pd
import numpy as np

# ^ 禁用同一单元格内的输出覆盖
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 两点之间的距离  

## angular separations

In [7]:
c1 = SkyCoord('5h23m34.5s', '-69d45m22s', frame='icrs')
c2 = SkyCoord('0h52m44.8s', '-72d49m43s', frame='fk5')  # * 注意到两个源所用的坐标系统不一致，计算时会自动转换
sep = c1.separation(c2)
sep

<Angle 20.74611448 deg>

**返回`astropy`的`Angle`数据类型**

In [8]:
type(sep)

astropy.coordinates.angles.Angle

将结果进行单位转换

In [10]:
# 将结果进行单位转换
sep.hour  # * 用小时表示
sep.arcminute  # * 用角分表示
sep.arcsecond  # * 用角秒表示

1.3830742984029318

1244.7668685626384

74686.0121137583

## distance

In [11]:
c1 = SkyCoord('5h23m34.5s', '-69d45m22s', distance=70*u.kpc, frame='icrs')
c2 = SkyCoord('0h52m44.8s', '-72d49m43s', distance=80*u.kpc, frame='icrs')
sep = c1.separation_3d(c2)
sep 

<Distance 28.74398816 kpc>

# Matching Catalogs

## 创建样例数据

In [28]:
# 创建样例
# * 待交叉的表
ra = [215.22238159,
 215.30378723,
 215.27708435,
 215.27868652,
 215.3032074]

dec = [53.00418472,
 53.05344391,
 53.03545761,
 53.03592682,
 53.05295563]
df = pd.DataFrame({'ra': ra, 'dec': dec})
df

# * 被交叉的表
ra = [215.27868652,
 215.3032074,
 215.26611328,
 215.22912598,
 215.2406311,
 215.27807617,
 215.24153137,
 215.26417542,
 215.29908752,
 215.30310059]

dec = [53.03592682,
 53.05295563,
 53.02712631,
 53.00312042,
 53.00996017,
 53.03676224,
 53.010746,
 53.02722168,
 53.05102539,
 53.05429459]

df_catalog = pd.DataFrame({'ra': ra, 'dec': dec})
df_catalog

,ra,dec
0,215.222382,53.004185
1,215.303787,53.053444
2,215.277084,53.035458
3,215.278687,53.035927
4,215.303207,53.052956


,ra,dec
0,215.278687,53.035927
1,215.303207,53.052956
2,215.266113,53.027126
3,215.229126,53.003120
4,215.240631,53.009960
5,215.278076,53.036762
6,215.241531,53.010746
7,215.264175,53.027222
8,215.299088,53.051025
9,215.303101,53.054295


## 交叉方式：找最近

In [31]:
ra1 = df['ra']
dec1 = df['dec']
ra2 = df_catalog['ra']
dec2 = df_catalog['dec']
c = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
catalog = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
idx # * 在被交叉表中找待交叉表中最近的源，在被交叉表中的索引
d2d # * 交叉后的两个源之间的角距离
d3d # * 交叉后的两个源之间的3D距离，assumes that all sources are at a distance of 1 (dimensionless)

array([3, 1, 0, 0, 1])

<Angle [0.00419576, 0.0005999 , 0.0010716 , 0.        , 0.        ] deg>

<Quantity [7.32298541e-05, 1.04702846e-05, 1.87029115e-05, 0.00000000e+00,
           0.00000000e+00]>

## 结果转换  
？没看懂

In [44]:
matches = catalog[idx]
(matches.separation(c) == d2d).all()  # * 校验匹配后两源之间的角距离
dra, ddec = c.spherical_offsets_to(matches)
dra
ddec

True

<Angle [ 4.05858187e-03, -3.48522261e-04,  9.63407455e-04,  0.00000000e+00,
         3.18055468e-15] deg>

<Angle [-1.06410922e-03, -4.88278591e-04,  4.69220763e-04,  0.00000000e+00,
         3.18055468e-15] deg>

In [46]:
from astropy.coordinates import match_coordinates_sky
idx, d2d, d3d = match_coordinates_sky(c, catalog)
idx
d2d
d3d
idx, d2d, d3d = match_coordinates_sky(c.frame, catalog.frame)

array([3, 1, 0, 0, 1])

<Angle [0.00419576, 0.0005999 , 0.0010716 , 0.        , 0.        ] deg>

<Quantity [7.32298541e-05, 1.04702846e-05, 1.87029115e-05, 0.00000000e+00,
           0.00000000e+00]>

# 交叉方式：设置阈值

In [65]:
max_sep = 3.0 * u.arcsec
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]
catalog_matches.ra.value
catalog_matches.dec.value

array([215.3032074 , 215.27868652, 215.3032074 ])

array([53.05295563, 53.03592682, 53.05295563])